<img width="8%" alt="Python.png" src="https://raw.githubusercontent.com/wirrywoo/ACubed/refs/heads/master/assets/logo/no-dark-mode.png" style="border-radius: 15%">

# Overview of Supported Datatypes
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/wirrywoo/acubed/issues">Bug report</a>

**Author:** [`WirryWoo`](https://www.flashflashrevolution.com/profile/WirryWoo/)

**Last update:** `November 5, 2024` (Created: `November 4, 2024`)

**Description:** This notebook provides a comprehensive overview of the datatypes supported in `acubed`.

**References:**
- [Python Currency converter](https://forex-python.readthedocs.io/en/latest/usage.html)

## Introduction

`acubed` provides a suite of modules designed to tackle challenges associated with standardizing stepfile difficulty measurement and assessing player skill rating in vertical scroll rhythm games. These modules operate on custom-built in-memory representations designed to facilitate the ingestion of stepfiles and player stats into a supported Python session for upstream machine learning analysis. It is essential for users to become acquainted with these representations, as preparing data in a format compatible with `acubed` is typically the initial step when utilizing any of its modules.

### Import libraries

In [1]:
from acubed.datatypes import Note, Stepfile

### Notes

In the realm of vertical scroll rhythm games, notes act as the fundamental "atoms" crucial to defining the gameplay experience. These elements appear as visual cues that generally move vertically across the screen toward designated receptors. Players must respond to these notes in synchronization with the music by pressing the appropriate keys or buttons.

Each note is characterized by specific attributes that differentiate it from others. The `time` attribute specifies when a player needs to hit the note as it reaches the receptors. The `step` attribute is a binary string representation that determines which orientations must be activated when the note aligns with the receptors. The precise configuration and timing of these notes are critical, as they influence the rhythm and difficulty of the game, offering both challenge and engagement for players. Understanding the structure of these note atoms is essential for analyzing rhythm game levels.

In `acubed`, you can create `Note` objects by specifying the `time` and `step` attributes. For example, in a $4$-key setup with the order of left, down, up, and right, pressing both the up and down arrows $2.5$ seconds into the level can be represented as:

In [2]:
Note(time = 2.5, step = '0110')

Note(time=2.5, step='0110')

A `ValueError` will be raised if one of two conditions are not met:

- The `time` attribute $t$ must be non-negative ($t \geq 0$).
- The `step` attribute $s$ must be a string composed solely of `0` or `1` for each orientation, with at least one of $n$ orientations having a value of `1`. Mathematically, this is expressed as $s \in \{(s_1, \cdots, s_n) \in \{0, 1\}^n | \sum_{i=1}^n s_i > 0 \}$.

In [3]:
Note(time = -2.5, step = '0110')

ValueError: Time attribute must be non-negative.

In [4]:
Note(time = 2.5, step = '0120')

ValueError: Step attribute must be one of ['0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111'].

To accommodate rhythm games with different numbers of receptors, such as the $6$-key Solo game mode in Etterna, `Note` objects can be instantiated by overriding the default `num_receptors` attribute to match the desired $n$-key setup.

In [5]:
Note(time = 2.5, step = '001100', num_receptors = 6)

Note(time=2.5, step='001100')

#### Supported Note Operations

In `acubed`, the addition operation for `Note` objects is implemented in a somewhat unconventional manner. This operation represents the following combinations:
- Merging two non-overlapping single steps results in a jump.
- Merging a non-overlapping step with a jump yields a hand.
- Merging two non-overlapping jumps, or a non-overlapping note with a hand, results in a quad.

Shown below is an example of adding one non-overlapping step with a jump to form a hand.

In [6]:
Note(time = 2.5, step = '0110') + Note(time = 2.5, step = '0001')

Note(time=2.5, step='0111')

`Note` objects can be added together only if they share the same `time` attribute and if their `step` attribute does not overlap (i.e. contains no zero-framers). We will address zero-framer processing in the next section.

In [7]:
Note(time = 2.5, step = '0110') + Note(time = 3, step = '0110')

ValueError: Cannot add two Notes with different times attributes.

In [8]:
Note(time = 2.5, step = '0110') + Note(time = 2.5, step = '0101')

ValueError: Cannot add Notes with overlapping step orientations.

### Stepfiles

TODO